# Imports

In [2]:
import numpy as np
import sys
import os
#%load_ext autoreload

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#from src.utils.audio_to_spectrograms import *
from src.models.yamnet_train import *

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    path_prefix = ''
else:
    path_prefix = '..'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

2025-12-03 01:40:46.749611: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-03 01:40:46.775890: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-03 01:40:47.567327: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 01:41:03.846205: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

Using: cpu


In [3]:
# manual train test split (stratified)
np.random.seed(0)
data = pickle.load(open('data/processed/yamnet/spectrograms_train.pkl', 'rb'))
train_size = 0.8
train_idx = []
for label in np.unique(data['event_label']):
    choices = np.where(data['event_label'] == label)[0]
    train_idx.append(np.sort(np.random.choice(choices, size = int(np.round(len(choices)*train_size)), replace = False)))
train_idx = np.sort(np.concatenate(train_idx))
len(train_idx)

2001

In [8]:
solver = Solver(epochs = 5, train_idx=train_idx, path_prefix = path_prefix, device = device)
solver.train()

Pre-computing features for 2500 files...


100%|██████████| 2500/2500 [00:03<00:00, 755.26it/s]


Epoch 1: train loss = 0.1359
Epoch 1: valildation loss = 0.1142
Epoch 2: train loss = 0.0889
Epoch 2: valildation loss = 0.0985
Epoch 3: train loss = 0.0727
Epoch 3: valildation loss = 0.0864
Epoch 4: train loss = 0.0624
Epoch 4: valildation loss = 0.0913
Epoch 5: train loss = 0.0532
Epoch 5: valildation loss = 0.1010
Epoch 6: train loss = 0.0458
Epoch 6: valildation loss = 0.1009
Epoch 7: train loss = 0.0383


2025-12-03 02:05:52.719 | INFO     | src.models.yamnet_train:train:218 - Saved model to checkpoints/yamnet_detector.pth


Epoch 7: valildation loss = 0.1110


In [9]:
events = solver.evaluate_full([i for i in range(len(data['event_label'])) if i not in train_idx])

Pre-computing features for 499 files...


Testing on data/processed/detection/train...: 100%|██████████| 499/499 [01:33<00:00,  5.33it/s]
